**Table of contents**<a id='toc0_'></a>    
- [This code changes the value labels and saves the updated dataframe to a new spsss .sav file](#toc1_1_1_1_)    
      - [This code does the following:](#toc1_1_1_2_)    
      - [Loop through rows and columns and generating a crosstab for each combination is working](#toc1_1_1_3_)    
      - [This code takes all the crosstab outputs and appends them to an excel sheet](#toc1_1_1_4_)    
      - [Appends are done as new columns instead of new rows so this isn't exactly what I wanted yet](#toc1_1_1_5_)    
      - [This append crosstab output as new rows for each crosstab row variable run instead of appending as additional columns](#toc1_1_1_6_)    
      - [The resulting excel file is easily manipulated to create data tables for use in heat maps like the PH reports](#toc1_1_1_7_)    
      - [This works.](#toc1_1_1_8_)    
      - [Crosstabs are generated and added to an excel file that is named for the row variable series](#toc1_1_1_9_)    
      - [Using this code in production](#toc1_1_1_10_)    
      - [Q3_r# series](#toc1_1_1_11_)    
      - [Q14 series](#toc1_1_1_12_)    
      - [This didn't generate output consistent with some of the crosstab runs I did with SPSS](#toc1_1_1_13_)    
      - [Figured it out!](#toc1_1_1_14_)    
      - [Recode for Q14 series did not include a label for response of 8 'I don't know'.](#toc1_1_1_15_)    
      - [I added a data label for response of 8 and the crosstab is working now.](#toc1_1_1_16_)    
      - [SPSS knew how to handle this but pandas did not so calculations were off in crosstab.](#toc1_1_1_17_)    
      - [This is the error: KeyError: "None of [Index(['Unchecked', 'Checked'], dtype='object', name='Q3_1')] are in the [index]"](#toc1_1_1_18_)    
      - [Q15 Series](#toc1_1_1_19_)    
      - [Q1_r# series](#toc1_1_1_20_)    
      - [09/26/2](#toc1_1_1_21_)    
      - [Clean up the code and make it more modular](#toc1_1_1_22_)    
      - [Use other modules that I have created to streamline the code](#toc1_1_1_23_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

#### <a id='toc1_1_1_1_'></a>[This code changes the value labels and saves the updated dataframe to a new spsss .sav file](#toc0_)

In [ ]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    print("Existing value labels for variable:", var_name)
    print(var_metadata)

    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    print("\nUpdated value labels for variable:", var_name)
    print(meta.variable_value_labels[var_name])

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    # pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
    pyreadstat.write_sav(df, new_meta_path)
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

df2 = df[df["Q3_1"] == 1]

# Save the updated metadata to a new file
new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
# pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
pyreadstat.write_sav(df2, new_meta_path)

filtered_df, meta2 = pyreadstat.read_sav(new_meta_path)
meta2_dict = dict(zip(meta2.column_names, meta2.column_labels))


crosstab_result = pd.crosstab(filtered_df['Q3_1']. \
            map(meta2.variable_value_labels['Q3_1']), \
            filtered_df['GENDER'].map(meta2.variable_value_labels['GENDER']), \
            dropna=True, normalize='columns'). \
            loc[meta2.variable_value_labels['Q3_1'].values()]. \
            loc[:,meta2.variable_value_labels['GENDER'].values()]*100


# filter the dataframe to return records that contain 'checked' for Q3_1
# filtered_df = df[df['Q3_1'] == 1]
# print(filtered_df.shape)

# crosstab_result = pd.crosstab(filtered_df['Q3_1'], filtered_df['GENDER'], normalize='columns')
print(crosstab_result)


In [ ]:
# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))


# Filter the dataframe to only return Q3_1 == 1
#filtered_df = df[df['Q3_1'] == 1]

pd.crosstab(df['Q3_1'].\
            map(meta.variable_value_labels['Q3_1']), \
            df['GENDER'].map(meta.variable_value_labels['GENDER']), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels['Q3_1'].values()]. \
            loc[:,meta.variable_value_labels['GENDER'].values()]*100

In [ ]:
df.shape

#### <a id='toc1_1_1_2_'></a>[This code does the following:](#toc0_)
- Reads in an SPSS dataset and metadata
- Modifies variable value labels to get rid of a "/" in the other/binary label
- Creates a dictionary to associate the dataset with the metadata
- Crosstabs a single row variable with a list of column variables
- Concats each crosstab output into a single output dataframe
- Prints the final output dataframe

In [ ]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
    # pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
    pyreadstat.write_sav(df, new_meta_path)
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

# list of variables to use as columns and rows for the crosstabs
row_variable = ['Q3_1']
column_variable = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
# variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']

crosstab_result_concat = pd.DataFrame()
# loop through the column variable list and crosstab each one
for col in column_variable:
    crosstab_result = pd.crosstab(df['Q3_1'].\
            map(meta.variable_value_labels['Q3_1']), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels['Q3_1'].values()]. \
            loc[:,meta.variable_value_labels[col].values()]*100
            
    # frames = [crosstab_result_concat, crosstab_result]

    crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    
    print(crosstab_result_concat)


#### <a id='toc1_1_1_3_'></a>[Loop through rows and columns and generating a crosstab for each combination is working](#toc0_)

In [ ]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
    # pyreadstat.write_sav(data_path, df, metadata=new_meta_path)
    pyreadstat.write_sav(df, new_meta_path)
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

# list of variables to use as columns and rows for the crosstabs
row_variable = ['Q3_1', 'Q3_2','Q3_3','Q3_4','Q3_5']
column_variable = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
# variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']

crosstab_result_concat = pd.DataFrame()
big_crosstab_result_concat = pd.DataFrame()
# loop through the column variable list and crosstab each one

# generate crosstabs for each combination of rows and columns
for row in row_variable:

    for col in column_variable:
        crosstab_result = pd.crosstab(df[row].\
            #map(meta.variable_value_labels['Q3_1']), \
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]*100
            
    # frames = [crosstab_result_concat, crosstab_result]
        print(crosstab_result)
       # filtered_df = crosstab_result[crosstab_result[row] == 'Checked']
        #print(filtered_df)

        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    
    print('crosstab_result_concat is:')
    print(crosstab_result_concat)

big_crosstab_result_concat = pd.concat([big_crosstab_result_concat, crosstab_result_concat])
print(big_crosstab_result_concat)

crosstab_result_concat.to_excel(f'../output_manipulation/output_folder/{row}_crosstab_result_concat.xlsx')


#### <a id='toc1_1_1_4_'></a>[This code takes all the crosstab outputs and appends them to an excel sheet](#toc0_)
#### <a id='toc1_1_1_5_'></a>[Appends are done as new columns instead of new rows so this isn't exactly what I wanted yet](#toc0_)

In [ ]:
import pandas as pd
import pyreadstat
import openpyxl

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
meta_dict = dict(zip(meta.column_names, meta.column_labels))

# list of variables to use as columns and rows for the crosstabs
row_variable = ['Q3_1', 'Q3_2','Q3_3','Q3_4','Q3_5']
column_variable = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
# variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']

# create the dataframe and excel file to save crosstab outputs to
crosstab_result_concat = pd.DataFrame()
crosstab_result_concat.to_excel('../output_manipulation/output_folder/crosstabs.xlsx')

# generate crosstabs for each combination of rows and columns
for row in row_variable:

    for col in column_variable:
        crosstab_result = pd.crosstab(df[row].\
            #map(meta.variable_value_labels['Q3_1']), \
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]*100
            
        print(f'crosstab result for {col} is: {crosstab_result}')

        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    with pd.ExcelWriter('../output_manipulation/output_folder/crosstabs.xlsx', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1')

    #print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

#### <a id='toc1_1_1_6_'></a>[This append crosstab output as new rows for each crosstab row variable run instead of appending as additional columns](#toc0_)
#### <a id='toc1_1_1_7_'></a>[The resulting excel file is easily manipulated to create data tables for use in heat maps like the PH reports](#toc0_)

In [1]:
import pandas as pd
import pyreadstat
import openpyxl

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

## This codeblock changes variable value labels                                   ##
## I had to do this because the old label used a '/' and was not python friendly  ##
# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
# I used this line because I saw it in a tutorial but the meta_dict object never gets called in this code
# so i commented it out
#meta_dict = dict(zip(meta.column_names, meta.column_labels))

# list of variables to use as columns and rows for the crosstabs
row_variable = ['Q3_1', 'Q3_2','Q3_3','Q3_4','Q3_5']
column_variable = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
# variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']

# create the dataframe and excel file to save crosstab outputs to
# crosstab_result_concat = pd.DataFrame()
df1 = pd.DataFrame()
df1.to_excel('../output_manipulation/output_folder/crosstabs.xlsx')
row_num = 0
col_num = 0

# generate crosstabs for each combination of rows and columns
for row in row_variable:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variable:
        crosstab_result = pd.crosstab(df[row].\
            #map(meta.variable_value_labels['Q3_1']), \
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')

        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    with pd.ExcelWriter('../output_manipulation/output_folder/crosstabs.xlsx', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=row_num, startcol=col_num)
    row_num += 3
    crosstab_result = []
    
    #print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

OSError: Cannot save file into a non-existent directory: '../output_manipulation/output_folder'

#### <a id='toc1_1_1_8_'></a>[This works.](#toc0_)
#### <a id='toc1_1_1_9_'></a>[Crosstabs are generated and added to an excel file that is named for the row variable series](#toc0_)

In [2]:
import pandas as pd
import pyreadstat
import openpyxl

# Import the data set using pyreadstat
data_path = '../../SPSS-Python/spss-datasets/PH_consumer_data_practice.sav'
df, meta = pyreadstat.read_sav(data_path)

# list of variables to use as columns and rows for the crosstabs
row_variables = ['Q3_1', 'Q3_2','Q3_3','Q3_4','Q3_5']
column_variables = ['GENDER', 'WhiteVNonWhite', 'AGEGROUP', 'INCOMEGROUP']
excel_file_name = f'{row_variables[0]}_series.xlsx'
print(excel_file_name)
output_path = f'../output_manipulation/output_folder/'
print(output_path)
print(output_path + excel_file_name)

## This codeblock changes variable value labels                                   ##
## I had to do this because the old label used a '/' and was not python friendly  ##
## Would have been easier to just change the label in SPSS and save as a new dataset ##
# Access the variable metadata
var_name = 'GENDER'
var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
if var_metadata is not None:
    
    # Define the new value labels
    new_value_labels = {
        1.0: 'Male',
        2.0: 'Female',
        3.0: 'Other_non_binary'
    }

    # Update the variable's value labels
    meta.variable_value_labels[var_name] = new_value_labels

    # Save the updated metadata to a new file
    new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
else:
    print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
# I used this line because I saw it in a tutorial but the meta_dict object never gets called in this code
# so i commented it out
#meta_dict = dict(zip(meta.column_names, meta.column_labels))

# create the dataframe and excel file to save crosstab outputs to
# crosstab_result_concat = pd.DataFrame()
df1 = pd.DataFrame()
df1.to_excel(f'{output_path + excel_file_name}')

# generate crosstabs for each combination of rows and columns
for row in row_variables:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variables:
        crosstab_result = pd.crosstab(df[row].\
            #map(meta.variable_value_labels['Q3_1']), \
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')
        
        # This section is to filter the output to only pass "Checked" responses for row variable
        # This works but output is formatted vertically vice horizontally
        # iloc method will fail if order of row variable value label is changed
        print('\n')
        selected_row = crosstab_result.iloc[-1]
        #selected_row = selected_row.T
        print('selected a row to keep')
        print(selected_row)

        # join the result of this loop to the consolidation dataframe
        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    reader = pd.read_excel(f'{output_path + excel_file_name}')
    with pd.ExcelWriter(f'{output_path + excel_file_name}', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=len(reader)+1)
    crosstab_result = []
    # with pd.ExcelWriter('../output_manipulation/output_folder/crosstabs.xlsx', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
    #     crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=row_num, startcol=col_num)
    # row_num += 3
    # crosstab_result = []
    
    #print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

Q3_1_series.xlsx
../output_manipulation/output_folder/
../output_manipulation/output_folder/Q3_1_series.xlsx


OSError: Cannot save file into a non-existent directory: '../output_manipulation/output_folder'

#### <a id='toc1_1_1_10_'></a>[Using this code in production](#toc0_)
#### <a id='toc1_1_1_11_'></a>[Q3_r# series](#toc0_)

In [4]:
import pandas as pd
import pyreadstat
import openpyxl

# Import the data set using pyreadstat
data_path = '../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/MCCU Report Draft and Templat_/9437 Financial Data_with_recodes_gender_labels_modified.sav'
df, meta = pyreadstat.read_sav(data_path)

# list of variables to use as columns and rows for the crosstabs
row_variables = ['Q3_r1_recode','Q3_r6_recode','Q3_r9_recode','Q3_r12_recode']
column_variables = ['Gender', 'Race_1', 'Hispanic', 'Race_2', 'Asian1','Qual1_recode','Qual7','BankingAccounts']
excel_file_name = f'{row_variables[0]}_series.xlsx'
print(excel_file_name)
output_path = f'../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/'
print(output_path)
print(output_path + excel_file_name)

## This codeblock changes variable value labels                                   ##
## I had to do this because the old label used a '/' and was not python friendly  ##
## Would have been easier to just change the label in SPSS and save as a new dataset ##
# Access the variable metadata
# var_name = 'GENDER'
# var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
# if var_metadata is not None:
    
#     # Define the new value labels
#     new_value_labels = {
#         1.0: 'Male',
#         2.0: 'Female',
#         3.0: 'Other_non_binary'
#     }

#     # Update the variable's value labels
#     meta.variable_value_labels[var_name] = new_value_labels

#     # Save the updated metadata to a new file
#     new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
# else:
#     print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
# I used this line because I saw it in a tutorial but the meta_dict object never gets called in this code
# so i commented it out
#meta_dict = dict(zip(meta.column_names, meta.column_labels))

# create the dataframe and excel file to save crosstab outputs to
# crosstab_result_concat = pd.DataFrame()
df1 = pd.DataFrame()
df1.to_excel(f'{output_path + excel_file_name}')

# generate crosstabs for each combination of rows and columns
for row in row_variables:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variables:
        crosstab_result = pd.crosstab(df[row].\
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')
        
        # This section is to filter the output to only pass "Checked" responses for row variable
        # This works but output is formatted vertically vice horizontally
        # iloc method will fail if order of row variable value label is changed
        # print('\n')
        # selected_row = crosstab_result.iloc[-1]
        #selected_row = selected_row.T
        # print('selected a row to keep')
        # print(selected_row)

        # join the result of this loop to the consolidation dataframe
        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    reader = pd.read_excel(f'{output_path + excel_file_name}')
    with pd.ExcelWriter(f'{output_path + excel_file_name}', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=len(reader)+1)
    crosstab_result = []
    # with pd.ExcelWriter('../output_manipulation/output_folder/crosstabs.xlsx', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
    #     crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=row_num, startcol=col_num)
    # row_num += 3
    # crosstab_result = []
    
    #print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

Q3_r1_recode_series.xlsx
../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/
../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/Q3_r1_recode_series.xlsx
crosstab result for Gender is: Gender            Male    Female
Q3_r1_recode                    
Experienced   0.095694  0.068729
Familiar      0.363636  0.319588
Unfamiliar    0.540670  0.611684
crosstab result for Race_1 is: Race_1        Unchecked   Checked
Q3_r1_recode                     
Experienced    0.089506  0.062500
Familiar       0.361111  0.295455
Unfamiliar     0.549383  0.642045
crosstab result for Hispanic is: Hispanic           Yes        No
Q3_r1_recode                    
Experienced   0.116564  0.062315
Familiar      0.398773  0.308605
Unfamiliar    0.484663  0.629080
crosstab result for Race_2 is: Race_2        Unchecked   Checked
Q3_r1_recode                     
Experienced    0.088889  0.057143
Familiar       0.344444  0.321429
Unfamiliar     0.566667  0.621429
crosstab result for Asian

#### <a id='toc1_1_1_12_'></a>[Q14 series](#toc0_)
#### <a id='toc1_1_1_13_'></a>[This didn't generate output consistent with some of the crosstab runs I did with SPSS](#toc0_)
#### <a id='toc1_1_1_14_'></a>[Figured it out!](#toc0_)
#### <a id='toc1_1_1_15_'></a>[Recode for Q14 series did not include a label for response of 8 'I don't know'.](#toc0_)
#### <a id='toc1_1_1_16_'></a>[I added a data label for response of 8 and the crosstab is working now.](#toc0_)
#### <a id='toc1_1_1_17_'></a>[SPSS knew how to handle this but pandas did not so calculations were off in crosstab.](#toc0_)

#### <a id='toc1_1_1_18_'></a>[This is the error: KeyError: "None of [Index(['Unchecked', 'Checked'], dtype='object', name='Q3_1')] are in the [index]"](#toc0_) [&#8593;](#toc0_)

In [6]:
import pandas as pd
import pyreadstat
import openpyxl

# Import the data set using pyreadstat
data_path = '../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/MCCU Report Draft and Templat_/9437 Financial Data_with_recodes_gender_labels_modified.sav'
df, meta = pyreadstat.read_sav(data_path)

# list of variables to use as columns and rows for the crosstabs
row_variables = ['Q14_r1_recode','Q14_r2_recode','Q14_r3_recode','Q14_r4_recode', 'Q14_r5_recode', 'Q14_r6_recode']
column_variables = ['Gender', 'Race_1', 'Hispanic', 'Race_2', 'Asian1','Qual1_recode','Qual7','BankingAccounts']
excel_file_name = f'{row_variables[0]}_series.xlsx'
# print(f'Excel file name = {excel_file_name}')
output_path = f'../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/'
# print(output_path)
print(f'Output file: {output_path + excel_file_name}')

## This codeblock changes variable value labels                                   ##
## I had to do this because the old label used a '/' and was not python friendly  ##
## Would have been easier to just change the label in SPSS and save as a new dataset ##
# Access the variable metadata
# var_name = 'GENDER'
# var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
# if var_metadata is not None:
    
#     # Define the new value labels
#     new_value_labels = {
#         1.0: 'Male',
#         2.0: 'Female',
#         3.0: 'Other_non_binary'
#     }

#     # Update the variable's value labels
#     meta.variable_value_labels[var_name] = new_value_labels

#     # Save the updated metadata to a new file
#     new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
# else:
#     print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
# I used this line because I saw it in a tutorial but the meta_dict object never gets called in this code
# so i commented it out
meta_dict = dict(zip(meta.column_names, meta.column_labels))

# create the dataframe and excel file to save crosstab outputs to
# crosstab_result_concat = pd.DataFrame()
df1 = pd.DataFrame()
df1.to_excel(f'{output_path + excel_file_name}')

# generate crosstabs for each combination of rows and columns
for row in row_variables:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variables:
        crosstab_result = pd.crosstab(df[row].\
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')
        
        # This section is to filter the output to only pass "Checked" responses for row variable
        # This works but output is formatted vertically vice horizontally
        # iloc method will fail if order of row variable value label is changed
        # print('\n')
        # selected_row = crosstab_result.iloc[-1]
        #selected_row = selected_row.T
        # print('selected a row to keep')
        # print(selected_row)

        # join the result of this loop to the consolidation dataframe
        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    reader = pd.read_excel(f'{output_path + excel_file_name}')
    with pd.ExcelWriter(f'{output_path + excel_file_name}', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=len(reader)+1)
    crosstab_result = []
    # with pd.ExcelWriter('../output_manipulation/output_folder/crosstabs.xlsx', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
    #     crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=row_num, startcol=col_num)
    # row_num += 3
    # crosstab_result = []
    
    #print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

Output file: ../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/Q14_r1_recode_series.xlsx
crosstab result for Gender is: Gender             Male    Female
Q14_r1_recode                    
Not T2B        0.297297  0.335938
T2B            0.702703  0.664062
crosstab result for Race_1 is: Race_1         Unchecked   Checked
Q14_r1_recode                     
Not T2B         0.259124  0.419162
T2B             0.740876  0.580838
crosstab result for Hispanic is: Hispanic            Yes        No
Q14_r1_recode                    
Not T2B        0.275362  0.339934
T2B            0.724638  0.660066
crosstab result for Race_2 is: Race_2         Unchecked   Checked
Q14_r1_recode                     
Not T2B         0.369231  0.181034
T2B             0.630769  0.818966
crosstab result for Asian1 is: Asian1         Unchecked  Checked
Q14_r1_recode                    
Not T2B         0.310096     0.48
T2B             0.689904     0.52
crosstab result for Qual1_recode is: Qual1_recode   

#### <a id='toc1_1_1_19_'></a>[Q15 Series](#toc0_)

In [2]:
import pandas as pd
import pyreadstat
# import openpyxl

# Import the data set using pyreadstat
data_path = '/Users/patescalona/Library/CloudStorage/OneDrive-Personal/Marketing Reports/CCR Reports/MCCU/Working/MCCU Report Draft and Templat_/9437 Financial Data_rev2.sav'
df, meta = pyreadstat.read_sav(data_path)

# list of variables to use as columns and rows for the crosstabs
row_variables = ['Q15_1','Q15_2','Q15_3','Q15_4', 'Q15_5', 'Q15_6', 'Q15_7', 'Q15_8']
column_variables = ['Gender', 'Race_1', 'Hispanic', 'Race_2', 'Asian1','Qual1_recode','Qual7','BankingAccounts']
excel_file_name = f'{row_variables[0]}_series.xlsx'
print(excel_file_name)
output_path = f'../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/'
print(output_path)
print(output_path + excel_file_name)

## This codeblock changes variable value labels                                   ##
## I had to do this because the old label used a '/' and was not python friendly  ##
## Would have been easier to just change the label in SPSS and save as a new dataset ##
# Access the variable metadata
# var_name = 'GENDER'
# var_metadata = meta.variable_value_labels.get(var_name)

# Check if the variable has value labels
# if var_metadata is not None:
    
#     # Define the new value labels
#     new_value_labels = {
#         1.0: 'Male',
#         2.0: 'Female',
#         3.0: 'Other_non_binary'
#     }

#     # Update the variable's value labels
#     meta.variable_value_labels[var_name] = new_value_labels

#     # Save the updated metadata to a new file
#     new_meta_path = "../output_manipulation/output_folder/gender_metadata_value_label_updated.sav"
    
# else:
#     print("Variable does not have value labels.")

# Create a dictionary so that we can selectively display the correct label for a column if necessary.
# I used this line because I saw it in a tutorial but the meta_dict object never gets called in this code
# so i commented it out
#meta_dict = dict(zip(meta.column_names, meta.column_labels))

# create the dataframe and excel file to save crosstab outputs to
# crosstab_result_concat = pd.DataFrame()
df1 = pd.DataFrame()
df1.to_excel(f'{output_path + excel_file_name}')

# generate crosstabs for each combination of rows and columns
for row in row_variables:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variables:
        crosstab_result = pd.crosstab(df[row].\
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')
        
        # This section is to filter the output to only pass "Checked" responses for row variable
        # This works but output is formatted vertically vice horizontally
        # iloc method will fail if order of row variable value label is changed
        # print('\n')
        # selected_row = crosstab_result.iloc[-1]
        #selected_row = selected_row.T
        # print('selected a row to keep')
        # print(selected_row)

        # join the result of this loop to the consolidation dataframe
        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    reader = pd.read_excel(f'{output_path + excel_file_name}')
    with pd.ExcelWriter(f'{output_path + excel_file_name}', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=len(reader)+1)
    crosstab_result = []
    # with pd.ExcelWriter('../output_manipulation/output_folder/crosstabs.xlsx', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
    #     crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=row_num, startcol=col_num)
    # row_num += 3
    # crosstab_result = []
    
    #print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

Q15_1_series.xlsx
../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/
../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/Q15_1_series.xlsx
crosstab result for Gender is: Gender        Male    Female
Q15_1                       
Unchecked  0.54067  0.553265
Checked    0.45933  0.446735
crosstab result for Race_1 is: Race_1     Unchecked   Checked
Q15_1                         
Unchecked   0.570988  0.505682
Checked     0.429012  0.494318
crosstab result for Hispanic is: Hispanic        Yes        No
Q15_1                        
Unchecked  0.582822  0.531157
Checked    0.417178  0.468843
crosstab result for Race_2 is: Race_2     Unchecked   Checked
Q15_1                         
Unchecked   0.538889  0.571429
Checked     0.461111  0.428571
crosstab result for Asian1 is: Asian1     Unchecked   Checked
Q15_1                         
Unchecked   0.552743  0.461538
Checked     0.447257  0.538462
crosstab result for Qual1_recode is: Qual1_recode     18-34     35-55 

#### <a id='toc1_1_1_20_'></a>[Q1_r# series](#toc0_)

In [9]:
import pandas as pd
import pyreadstat

# Import the data set using pyreadstat
data_path = '../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working//MCCU Report Draft and Templat_/9437 Financial Data_rev2.sav'
df, meta = pyreadstat.read_sav(data_path)

# list of variables to use as columns and rows for the crosstabs
row_variables = ['Q1_r1','Q1_r2','Q1_r3']
column_variables = ['Gender', 'Race_1', 'Hispanic', 'Race_2', 'Asian1','Qual1_recode','Qual7','BankingAccounts']

# specify file path, name, and workbook sheet 
output_path = f'../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/'
excel_file_name = f'{row_variables[0]}_series.xlsx'
wb_sheet_name = f'{row_variables[0]}'
print(output_path + excel_file_name)

# create the dataframe and excel file to save crosstab outputs
df1 = pd.DataFrame()
df1.to_excel(f'{output_path + excel_file_name}')

# generate crosstabs for each combination of rows and columns
for row in row_variables:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variables:
        crosstab_result = pd.crosstab(df[row].\
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')

        # join the result of this loop to the consolidation dataframe
        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    reader = pd.read_excel(f'{output_path + excel_file_name}')
    with pd.ExcelWriter(f'{output_path + excel_file_name}', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name=wb_sheet_name, startrow=len(reader)+1)
    crosstab_result = []
    
#print(f'crosstab_result_concat for {row} is:')
print(crosstab_result_concat)

../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/Q1_r1_series.xlsx
crosstab result for Gender is: Gender                                   Male    Female
Q1_r1                                                  
Currently use                        0.818182  0.752577
Have used in the past                0.086124  0.092784
Have considered but have never used  0.009569  0.013746
Have never considered                0.086124  0.140893
crosstab result for Race_1 is: Race_1                               Unchecked   Checked
Q1_r1                                                   
Currently use                         0.820988  0.704545
Have used in the past                 0.061728  0.142045
Have considered but have never used   0.015432  0.005682
Have never considered                 0.101852  0.147727
crosstab result for Hispanic is: Hispanic                                  Yes        No
Q1_r1                                                  
Currently use                    

#### <a id='toc1_1_1_21_'></a>[09/26/2](#toc0_)
#### <a id='toc1_1_1_22_'></a>[Clean up the code and make it more modular](#toc0_)
#### <a id='toc1_1_1_23_'></a>[Use other modules that I have created to streamline the code](#toc0_)

In [1]:
import pandas as pd
import pyreadstat
# use save_to_excel.py module
import save_to_excel

# Import the data set using pyreadstat
data_path = '../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/MCCU Report Draft and Templat_/9437 Financial Data_with_recodes_gender_labels_modified.sav'
df, meta = pyreadstat.read_sav(data_path)
df_with_metadata = pyreadstat.set_value_labels(df, meta)

# list of variables to use as columns and rows for the crosstabs
row_variables = ['Q14_r1_recode','Q14_r2_recode','Q14_r3_recode','Q14_r4_recode', 'Q14_r5_recode', 'Q14_r6_recode']
column_variables = ['Gender', 'Race_1', 'Hispanic', 'Race_2', 'Asian1','Qual1_recode','Qual7','BankingAccounts']
excel_file_name = 'MCCU_output.xlsx'
worksheet_name = f'{row_variables[0]}_series'
# print(f'Excel file name = {excel_file_name}')
output_path = f'../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/'
# print(output_path)
print(f'Output file: {output_path + excel_file_name}')


# meta_dict = dict(zip(meta.column_names, meta.column_labels))

# create the dataframe and excel file to save crosstab outputs to
df1 = pd.DataFrame()
df1.to_excel(f'{output_path + excel_file_name}')

# generate crosstabs for each combination of rows and columns
for row in row_variables:
    crosstab_result_concat = pd.DataFrame()

    for col in column_variables:
        crosstab_result = pd.crosstab(df[row].\
            map(meta.variable_value_labels[row]), \
            df[col].map(meta.variable_value_labels[col]), \
            dropna=True, normalize='columns'). \
            loc[meta.variable_value_labels[row].values()]. \
            loc[:,meta.variable_value_labels[col].values()]
            
        print(f'crosstab result for {col} is: {crosstab_result}')
        
        # join the result of this loop to the consolidation dataframe
        crosstab_result_concat = pd.concat([crosstab_result_concat, crosstab_result], axis=1)
    print(f'crosstab_result_concat for {col} is: \n ', crosstab_result_concat)

    # Append crosstab_result_concat to existing xlsx file here
    reader = pd.read_excel(f'{output_path + excel_file_name}')
    with pd.ExcelWriter(f'{output_path + excel_file_name}', engine='openpyxl', if_sheet_exists='overlay', mode='a') as writer:
        crosstab_result_concat.to_excel(writer, sheet_name='Sheet1', startrow=len(reader)+1)
    crosstab_result = []

print(crosstab_result_concat)

save_to_excel.open_excel_workbook(excel_file_name)
save_to_excel.save_result_to_xlworksheet(excel_file_name, worksheet_name, crosstab_result_concat)

Output file: ../../../../OneDrive/Marketing Reports/CCR Reports/MCCU/Working/MCCU_output.xlsx
crosstab result for Gender is: Gender             Male    Female
Q14_r1_recode                    
Not T2B        0.297297  0.335938
T2B            0.702703  0.664062
crosstab result for Race_1 is: Race_1         Unchecked   Checked
Q14_r1_recode                     
Not T2B         0.259124  0.419162
T2B             0.740876  0.580838
crosstab result for Hispanic is: Hispanic            Yes        No
Q14_r1_recode                    
Not T2B        0.275362  0.339934
T2B            0.724638  0.660066
crosstab result for Race_2 is: Race_2         Unchecked   Checked
Q14_r1_recode                     
Not T2B         0.369231  0.181034
T2B             0.630769  0.818966
crosstab result for Asian1 is: Asian1         Unchecked  Checked
Q14_r1_recode                    
Not T2B         0.310096     0.48
T2B             0.689904     0.52
crosstab result for Qual1_recode is: Qual1_recode      18-34 